In [1]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
import scipy as sp
sns.set_context('paper')
np.set_printoptions(suppress=True)
# pd.set_option('display.max_columns', None)

In [2]:
def simu_success(tag, params):
    # which task
    if tag == "Asso-CR":
        path = "../Data/simuS1_group1_design.pkl"
        nitems = 96
        test1_num = 40
    elif tag == "Pair-CR":
        path = "../Data/simuS1_group2_design.pkl"
        nitems = 176
        test1_num = 80
    elif tag == "Item-CR":
        path = "../Data/simuS1_group3_design.pkl"
        nitems = 136
        test1_num = 80

    # load stimuli
    with open(path, 'rb') as inp:
        df_study = pickle.load(inp)
        df_test = pickle.load(inp)
    # df_study = df_study.loc[df_study.session < 200]
    # df_test = df_test.loc[df_test.session < 200]

    # load semantic matrix
    s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

    # update n_item in params to fit the study
    params.update(nitems_in_accumulator = nitems)
    # print(params)

    # run CMR
    df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params, df_study, df_test ,s_mat)
    df_simu['test'] = df_test['test']
    df_simu = df_simu.merge(df_test,on=['session','test','test_itemno1','test_itemno2'])

    # get f_in
    sessions = np.unique(df_simu.session)
    tmp_corr_fin = []
    tmp_omax_fin = []

    for sess in sessions:
        df_tmp = df_study.loc[df_study.session == sess]
        tmp1 = df_tmp.study_itemno1.to_numpy()
        tmp2 = df_tmp.study_itemno2.to_numpy()
        df_tmp2 = df_test.loc[df_test.session == sess]
        tmp3 = df_tmp2.test_itemno1[df_tmp2.test_itemno1 >= 0].to_numpy()
        tmp4 = df_tmp2.test_itemno2[df_tmp2.test_itemno2 >= 0].to_numpy()
        tmp = np.concatenate((tmp1, tmp2, tmp3, tmp4))
        tmp = np.unique(tmp)  # sort

        tmp_corr = df_simu.loc[df_simu.session == sess,"correct_ans"][test1_num:]
        corrid = np.searchsorted(tmp, tmp_corr)

        corr_fin = [f_in_dif[sess][i][id] for i, id in enumerate(corrid)]
        omax_fin = [np.max(np.delete(f_in_dif[sess][i], id)) for i, id in enumerate(corrid)]

        tmp_corr_fin = tmp_corr_fin + [-1] * test1_num + corr_fin
        tmp_omax_fin = tmp_omax_fin + [-1] * test1_num + omax_fin

    df_simu['corr_fin'] = tmp_corr_fin
    df_simu['omax_fin'] = tmp_omax_fin

    # optimal threshold for test1
    csim_two = df_simu.query("test==1").groupby("correct_ans").csim.mean()
    opt_thresh = np.mean(csim_two)
    df_simu['s_resp'] = df_simu.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

    return df_simu

In [3]:
def anal_perform(df_simu):

    # get correctness
    df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans

    # recognition performance
    df_recog = df_simu.query("test==1")
    hr_far = df_recog.groupby("correct_ans")["s_resp"].mean().to_frame(name="Yes rate")
    hr = hr_far['Yes rate'][0]
    far = hr_far['Yes rate'][1]
    z_hr_far = sp.stats.norm.ppf(hr_far)
    d_prime = z_hr_far[1].item() - z_hr_far[0].item()
    print("recognition: \n", hr_far)
    print("d_prime: ", d_prime)

    # cued recall performance
    df_cr = df_simu.query("test==2")
    p_rc = df_cr.correct.mean()
    print("cued recall: \n", p_rc)

    # analyze pair
    def get_pair(df_tmp):
        df_tmp_pair = pd.pivot_table(df_tmp,index="pair_idx",columns="test",values="correct")
        df_tmp_pair.columns = ["test1","test2"]
        df_tmp_pair.reset_index(inplace=True)
        return df_tmp_pair

    df_simu_p = df_simu.query("pair_idx >= 0")
    df_pair = df_simu_p.groupby("session").apply(get_pair).reset_index()
    df_tab = pd.crosstab(index=df_pair.test2,columns=df_pair.test1,normalize=False)
    print("contingency table: \n", df_tab)
    print("contingency table norm: \n", pd.crosstab(index=df_pair.test2,columns=df_pair.test1,normalize='all'))

    # compute" Q
    def Yule_Q(A, B, C, D):
        return (A * D - B * C) / (A * D + B * C)
    q = Yule_Q(df_tab[1][1]+0.5,df_tab[0][1]+0.5,df_tab[1][0]+0.5,df_tab[0][0]+0.5)  # add 0.5
    print("Q: ", q)

    return p_rc, hr, far, d_prime, q

### Study Params

In [4]:
# study params should be the same across three groups
params = cmr.make_default_params()
params.update(
    beta_enc = 0.3,
    gamma_fc = 0.2,
    gamma_cf = 0.2,
    s_fc = 0.2,
    s_cf = 0.2,
    phi_s = 1,
    phi_d = 0.6,
    d_ass = 1,
    learn_while_retrieving = False,
    var_enc = 1,
    bad_enc_ratio = 1,
)

In [5]:
params.update(
    beta_cue = 0.4,
    beta_rec = 0.1,  # beta for retrieved item
    beta_distract = 0.1,
    # beta_rec_post = 0.1,
    c_thresh = 0.01,
    c_thresh_ass = 0.5,
    kappa = 0.01,
    lamb = 0.002,
    eta = 0.002,
    omega = 3,
    alpha = 1
)
params

{'beta_enc': 0.3,
 'beta_rec': 0.1,
 'beta_cue': 0.4,
 'beta_rec_post': 0.5,
 'beta_distract': 0.1,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 0.2,
 's_fc': 0.2,
 'kappa': 0.01,
 'eta': 0.002,
 'omega': 3,
 'alpha': 1,
 'c_thresh': 0.01,
 'c_thresh_ass': 0.5,
 'd_ass': 1,
 'lamb': 0.002,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 50,
 'max_recalls': 50,
 'learn_while_retrieving': False,
 'a': 2800,
 'b': 20,
 'm': 0,
 'n': 1,
 'c1': 0,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.2,
 'gamma_cf': 0.2}

### Association - CR (Group1)

In [8]:
tag = "Asso-CR"
g1_stats = []
for _ in range(10):
    df_simu_g1 = simu_success(tag, params)
    p_rc, hr, far, d_prime, q = anal_perform(df_simu_g1)
    g1_stats.append([p_rc, hr, far, d_prime, q])

ValueError: unsupported pickle protocol: 5

In [7]:
print(np.mean(np.array(g1_stats),axis=0))
print(sp.stats.sem(np.array(g1_stats),axis=0))

nan
nan


/home1/beigejin/.conda/envs/jinbeige/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home1/beigejin/.conda/envs/jinbeige/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home1/beigejin/.conda/envs/jinbeige/lib/python3.7/site-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/home1/beigejin/.conda/envs/jinbeige/lib/python3.7/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in true_divide
  subok=False)
/home1/beigejin/.conda/envs/jinbeige/lib/python3.7/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


### Pair - CR (Group2)

In [60]:
tag = "Pair-CR"
g2_stats = []
for _ in range(10):
    df_simu_g2 = simu_success(tag, params)
    p_rc, hr, far, d_prime, q = anal_perform(df_simu_g2)
    g2_stats.append([p_rc, hr, far, d_prime, q])

CMR2 Time: 148.01835536956787
recognition: 
              Yes rate
correct_ans          
0               0.022
1               0.975
d_prime:  3.972707054086244
cued recall: 
 0.388
contingency table: 
 test1  0.000  1.000
test2              
0.000    961  23519
1.000     30  15490
contingency table norm: 
 test1  0.000  1.000
test2              
0.000  0.024  0.588
1.000  0.001  0.387
Q:  0.9081002382832791
CMR2 Time: 152.3754904270172
recognition: 
              Yes rate
correct_ans          
0               0.022
1               0.975
d_prime:  3.972707054086244
cued recall: 
 0.3844
contingency table: 
 test1  0.000  1.000
test2              
0.000    968  23656
1.000     23  15353
contingency table norm: 
 test1  0.000  1.000
test2              
0.000  0.024  0.591
1.000  0.001  0.384
Q:  0.9279223093400225
CMR2 Time: 152.41817784309387
recognition: 
              Yes rate
correct_ans          
0               0.022
1               0.975
d_prime:  3.972707054086244
cued recall: 
 

In [61]:
print(np.mean(np.array(g2_stats),axis=0))
print(sp.stats.sem(np.array(g2_stats),axis=0))

[0.3862625  0.022275   0.975225   3.97270705 0.91078311]
[0.00065995 0.         0.         0.         0.00432642]


### Item - CR (Group3)

In [62]:
tag = "Item-CR"
g3_stats = []
for _ in range(10):
    df_simu_g3 = simu_success(tag, params)
    p_rc, hr, far, d_prime, q = anal_perform(df_simu_g3)
    g3_stats.append([p_rc, hr, far, d_prime, q])

CMR2 Time: 122.62793517112732
recognition: 
              Yes rate
correct_ans          
0               0.195
1               0.831
d_prime:  1.8186997653967534
cued recall: 
 0.4299
contingency table: 
 test1  0.000  1.000
test2              
0.000   5612  17192
1.000   1142  16054
contingency table norm: 
 test1  0.000  1.000
test2              
0.000  0.140  0.430
1.000  0.029  0.401
Q:  0.6420458628919693
CMR2 Time: 123.06224775314331
recognition: 
              Yes rate
correct_ans          
0               0.195
1               0.831
d_prime:  1.8186997653967534
cued recall: 
 0.4257
contingency table: 
 test1  0.000  1.000
test2              
0.000   5645  17327
1.000   1109  15919
contingency table norm: 
 test1  0.000  1.000
test2              
0.000  0.141  0.433
1.000  0.028  0.398
Q:  0.6475684058735289
CMR2 Time: 123.41632628440857
recognition: 
              Yes rate
correct_ans          
0               0.195
1               0.831
d_prime:  1.8186997653967534
cued recal

In [63]:
print(np.mean(np.array(g3_stats),axis=0))
print(sp.stats.sem(np.array(g3_stats),axis=0))

[0.4291     0.1949     0.83115    1.81869977 0.64778702]
[0.00082039 0.         0.         0.         0.00146871]
